In [1]:
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

import re
from string import punctuation
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
path_corpus = Path('../data/corpora/tutby_126784.csv')
path_corpus_clean = Path('../data/corpora/clean_tutby_126784_header.csv')

column_name = 'header'  # 'document' 'header'

In [3]:
%%time

data = pd.read_csv(path_corpus)

corpus = data[column_name]

print(data.shape)
display(data.head(3))

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,['министерства']


Wall time: 9.23 s


In [4]:
russian_stop_words = set(stopwords.words('russian'))
mystem = Mystem() 

def preprocess_text(text):
    text = text.lower()
    text = re.sub(u'\xa0|\n', ' ', text)
    text = re.sub('[^а-яa-z ]', '', text)
    # removing extra spaces...
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if ((token not in russian_stop_words) and (token.strip() not in punctuation) and (len(token) > 2))]
    text = ' '.join(tokens)
    return text

In [5]:
%%time

separator = 'qsefthukoaxdvgnjthukoaxdvefcngwojd'
batch_size = 1000

result = []

for i in tqdm(range(len(corpus) // batch_size + 1)):
    batch = corpus[i * batch_size:(i + 1) * batch_size]
    batch_joint = f' {separator} '.join(batch)
    batch_clean = preprocess_text(batch_joint)
    batch_clean = batch_clean.split(separator)
    result += batch_clean
    
result = [_.strip() for _ in result]


print(len(corpus), len(result))
print(corpus.iloc[-77])
print(result[-77])

data[column_name] = result
data.to_csv(path_corpus_clean, index=False)


126784 126784
«Хорошая кошка» от Great Wall: под брендом Ora представлен еще один электрокар хороший кошка great wall бренд ora представлять электрокар
Wall time: 3min 46s


In [6]:
data = pd.read_csv(path_corpus_clean)
print(data.shape)
display(data.head())

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,тренер шахтер оправдываться хотеть весь вопрос...,2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,['футбол']
1,https://news.tut.by/550307.html,Общество,зацветать каменный роза комсомольский вновь,2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,фотофакт скамейка вид пожарный машина появлять...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,['министерства']
3,https://news.tut.by/550309.html,Футбол,станислав драгун дебютировать бате матч жальгирис,2017-07-06T22:11:00+03:00,Чемпион Беларуси БАТЭ воспользовался паузой в ...,"['футбол', 'БАТЭ']"
4,https://news.tut.by/550310.html,В мире,генпрокурор украина пообещать открывать уголов...,2017-07-06T22:28:00+03:00,Генпрокуратура Украины откроет уголовное произ...,"['Ситуация в Украине', 'государственные перево..."
